# Meta 

* 이 노트북의 자료는 [anarinsk/simsangjung](https://github.com/anarinsk/simsangjung)에서 찾으실 수 있습니다. 
* 어차피 심상정 의원이 블로그에 올리신 내용이므로 큰 문제는 없으리라고 봅니다. 

# Preprocess Data 

* 이 섹션에서는 데이터 전처리를 다룬다. 별로 복잡한 코드는 없으므로 코드에 대한 설명을 생략하도록 하겠다.
* 마지막에 최종 결과를 df로 넘기게 되는데, pivot style을 얻으려면, `make_full_df(False)`를 melt style을 얻으로면, `make_full_df(true)`를 실행하면 된다. 
* 데이터는 깃허브에서 바로 읽도록 만들었다. 만일 로컬에서 실행하고 싶다면 적절하게 바꾸시길 바란다.  

In [1]:
# Import packages 
import pandas as pd 
import numpy as np 
from pathlib import Path 

In [5]:
# Set cwd in github 
#this_cwd = Path('/home/anarinsk/github/simsangjung') 
this_cwd = 'http://raw.githubusercontent.com/anarinsk/simsangjung/master/raw_data/'
# 0: 이자, 1: 배당 
baedang_ija_2016 = this_cwd + 'BaeDangSoDeuk+IJaSoDeuk+CheonBunWi.xlsx'
# 0: 2015년, 1: 2014년, 2: 2013년 
geunro_2013_2015 = this_cwd + 'GeunRoSoDeukCheonBunWi.xlsx'
geunro_2016 = this_cwd + 'GeunRoSoDeukCheonBunWi2016Nyeon.xlsx'

# Load into padas DataFrame 

def load_xlsx(filename, sheet, col_names, skip=5):
    return pd.read_excel(filename, sheet_name=sheet, names=col_names, 
                  skiprows=skip, index_col=None, header=None)

#pd.read_excel(baedang_ija_2016, sheet_name=0, skiprows=0, index_col=None, header=None)
#pd.read_excel(geunro_2016, sheet_name=0, skiprows=0, index_col=None, header=None)

df1 = load_xlsx(baedang_ija_2016, 0, ['quintile', 'n', 'income', 'tax'])
df2 = load_xlsx(baedang_ija_2016, 1, ['quintile', 'n', 'income', 'tax'])
df3 = load_xlsx(geunro_2016, 0, ['quintile', 'n', 'total','income', 'dedection', 'standard', 'tax'])
df4 = load_xlsx(geunro_2013_2015, 0, ['quintile', 'n', 'total','income', 'dedection', 'standard', 'tax'])
df5 = load_xlsx(geunro_2013_2015, 1, ['quintile', 'n', 'total','income', 'dedection', 'standard', 'tax'])
df6 = load_xlsx(geunro_2013_2015, 2, ['quintile', 'n', 'total','income', 'dedection', 'standard', 'tax'])

In [ ]:
q = 'http://raw.githubusercontent.com/anarinsk/simsangjung/master/raw_data/BaeDangSoDeuk+IJaSoDeuk+CheonBunWi.xlsx'
# 0: 이자, 1: 배당 
q / 'test'

In [ ]:
# Define some functions 

def my_tidy(df, year, cat, do_melt=False): 
    df['year'] = year 
    df['cat'] = cat 
    df['quintile'] = range(1,1001, 1)   
    
    if do_melt == False:
        return df
    else: 
        return pd.melt(df, id_vars=['quintile', 'year', 'cat'])
    
def make_full_df(do_melt=False): 
    res = [my_tidy(d, y, c, do_melt) for d,y,c in 
               [[df1, 2016, "interest"], [df2, 2016, "dividends"], 
                [df3, 2016, "work"], [df4, 2015, "work"], [df5, 2014, "work"], [df6, 2013, "work"]]
        ]
    return pd.concat(res)

In [ ]:
df_melt = make_full_df(True)
df_melt.reset_index(inplace=True) 

df_pivot = make_full_df(False)
df_pivot.reset_index(inplace=True) 

# save to csv 


df_melt.to_csv(this_cwd/'csv'/'melt.csv')
df_pivot.to_csv(this_cwd/'csv'/'pivot.csv')

# Some Poor EDA(Explantory Data Analysis) 

* 그래프는 R의 ggplot2를 임포트해서 그렸다. 몇가지 코멘트는 아래 코드와 결과 사이에 남기도록 한다. 
* 조세자료지만, 일단 소득원(income 컬럼)에 관해서면 살펴보았다. 조세 자료에 관한 보다 세밀한 분석은 나중에 누군가가 더 잘 해주겠지. 

In [ ]:
df.info()

In [ ]:
### Start with kernel withr 
import warnings
warnings.filterwarnings('ignore')
import os
import rpy2.rinterface
os.environ.get("R_HOME")
#os.environ['R_HOME'] = '/usr/lib/R'
%reload_ext rpy2.ipython

In [ ]:
%R librarian::shelf(tidyverse, DescTools, ggsci, ggthemes, showtext)

In [ ]:
%%R -i df
readRDS('./rds/df.rds') %>% 
  select(-index) %>% 
  group_by(year, cat) %>% 
  mutate(
    q1000 = as.integer(quintile), 
    q100 = cut(quintile, breaks = 100) %>% as.integer(),
    q10 = cut(quintile, breaks = 10) %>% as.integer()
    ) %>% 
  select(-quintile) -> tbl 

tbl %>% 
  filter(variable == 'income') %>% 
  group_by(year, cat) %>% 
  summarise(sum_yearcat = sum(value)) -> tbl2

tbl %>% 
  filter(variable == 'n') %>% 
  select(q1000, year, cat, value) %>% 
  rename(n = value) -> tbl3

tbl %>% 
  filter(variable == "income") %>% 
  left_join(tbl2, by = c("year", "cat")) %>% 
  left_join(tbl3, by = c('q1000', 'year', 'cat')) %>% 
  select(-variable) %>% 
  rename(income = value) -> income 

In [ ]:
%%R
df_by_q <- function(var,is_work, df=income){
  
  if(is_work == T){
    df <- df %>% filter(cat == "work")
  } else {
    df <- df %>% filter(year == 2016)
  }
  var_enq = enquo(var)
  df %>% 
    group_by(year, cat, !!var_enq) %>% 
    summarise(
      value = sum(income), 
      share = value / first(sum_yearcat),  
      mean = value / sum(n)
    )
}
#
generate_2016_q <- function(var){
  
  var_enq <- enquo(var)
  str_extract_all(deparse(substitute(var)), "[0-9]") %>% 
    unlist() %>%  
    str_c(collapse="") -> title
  
  showtext_auto() 
  
  df_by_q(!!var_enq, F) %>% 
  filter(!!var_enq %in% seq_len(10)) %>% 
  ggplot() + 
  aes(x = !!var_enq, y = share * 100) + 
  geom_col(width = 0.7) +
  scale_x_discrete(limits=as.character(seq_len(10))) +
  facet_grid(rows = vars(cat)) + 
  labs(
    title = str_glue("2016년 소득원천별 점유율 (상위 1-10분위, {title}분위 기준)"),  y = "%", x = "분위") + 
  theme_bw() +
  theme(text=element_text(size=10,  family="ng"))
}
#
generate_work_q <- function(var, q = seq_len(10)){
  var_enq <- enquo(var)
  str_extract_all(deparse(substitute(var)), "[0-9]") %>% 
    unlist() %>%  
    str_c(collapse="") -> title
  df_by_q(!!var_enq, T) %>% 
    filter(!!var_enq %in% q) %>% 
    ggplot() + 
    aes(x = year, y = share, fill = factor(!!var_enq)) + 
    geom_col(width = 0.7) + 
    scale_x_discrete(limits=c(2013,2014,2015,2016)) + 
    theme_bw() + 
    scale_fill_simpsons(name = "분위") + 
    labs(
      title = str_glue("근로소득 {title}분위 분배율 변화 (2013--2016)"),
      x = "연도", 
      y = "누적 %"
    )
}

font_add_google("Nanum Gothic", "ng")

# 분위별 분석 

* 심상정 의원의 자료로 살펴볼 수 있는 축은 두가지다. 
    * 하나는 배당소득, 이자소득, 근로소득을 비교하는 것이다. 이 자료는 2016년 자료만 제공되었다. 
    * 다른 하나는 2013년부터 2016년까지 4개 연도의 근로 소득을 비교하는 것이다. 

## 2016년 소득 종류별 비교 

* 일단 백분위를 십분위로 압축해서 종류별 소득의 규모를 살펴보자. 
* 아래 표에서 dividends는 배당소득, interest는 이자소득 그리고 work는 근로소득을 의미한다. 
* q1000은 천분위(원자료),  q100은 백분위 그리고 q10는 10분위로 자료를 재정리한 것이다. 
    * 작은 숫자일수록 많은 소득을 누리는 분위를 의미한다.  
* income은 각 종류별 소득의 분위합, n은 해당 구간의 샘플 수다. 
* n이 종류별로 각기 다르기 때문에 평균을 같이 살펴보았다. 예상대로 배당소득의 크기가 어마어마하고 이자소득의 크기 역시 상당하다. 

In [ ]:
%%R
income %>% 
  filter(year == 2016) %>% 
  group_by(cat, q10) %>%
  filter(q10 %in% c(1,2,3,4,5)) %>% 
  summarise(
    income2 = sum(income), 
    n2 = sum(n), 
    mean = (income2/n2) %>% sprintf("%#.2f", .)
  ) %>% 
  rename(
    income = income2, 
    n = n2
  ) %>% 
  knitr::kable()

### 1000분위,100분위, 10분위 기준 원천별 소득 점유율 비교 

* 아래 그림들을 보시면 명확하다. 상위 0.1%(1/1000)가 2016년 배당소득의 50%이상을 차지하고 있다. 1%는 약 85%, 10%는 약 95%다.  

In [ ]:
%%R
generate_2016_q(q1000)

In [ ]:
%%R 
generate_2016_q(q100)

In [ ]:
%%R 
generate_2016_q(q10)

### 지니계수 

* 각 소득원천별로 지니계수를 계산해 보자. 
* 근로소득에 비해서 배당소득과 이자소득은 '압도적으로' 불평등하다. 플로우의 누적합인 스톡, 즉 자산 불평등을 계산해 봤으면 싶은 마음이 드는 대목이다. 

In [ ]:
%%R 
df_by_q(q1000, F) %>% 
  summarise(
    gini = Gini(value) %>% sprintf("%#.2f", .)
  ) %>% 
  knitr::kable()

## 노동 소득의 변화 

* 시각화했을 때 100분위, 1000분위는 다소 애로사항이 있다. 솔루션을 찾을 수 있다면 추후에 다시 해보도록 하자. 
* 아래는 근로소득의 10분위 분배의 변화를 시각화한 것이다.
* 일단 2013년에서 2014년 사이에 하위 분위에서 점유율 증가가 눈에 들어온다. 통계작성의 변화 등이 없었다면, 꽤 주목할만한 변화다. 
    * 박근혜 정권이라고 나쁜 일만 했겠는가...
    * 좌간 하위 계층 복지에 있어서 뭔가 큰 변화가 감지되는 시기인 듯. 

In [ ]:
%%R
generate_work_q(q10)

* 하위 5개 분위의 변화를 보다 자세하게 보자.  5~10 분위 모두 고르게 증가했음을 알 수 있다. 

In [ ]:
%%R
generate_work_q(q10, seq(5,10,1))

* 노동 소득의 지니계수 변화 역시 같은 방향을 보여준다. 

In [ ]:
%%R
df_by_q(q1000, T) %>% 
  summarise(
    gini = Gini(value) %>% sprintf("%#.2f", .)
  ) %>% 
  knitr::kable()